# Predicting the Sensex with Political News

In [0]:
!pip install keras==2.0.4

    100% |████████████████████████████████| 204kB 30.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/92/11/19/2b4dda9f2301c140d1ac59460321c5be612c7cde4f26a8b8be
Successfully built keras
textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.4 which is incompatible.
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [0]:
import keras

Using TensorFlow backend.


In [0]:
 print(keras.__version__)

2.0.4


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import datetime
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score as acc
import matplotlib.pyplot as plt
from keras.legacy.layers import Merge
from keras.models import Sequential
from tensorflow.keras.initializers import TruncatedNormal 
from keras.layers import Dropout, Activation, Embedding, Convolution1D, MaxPooling1D, Input, Dense, \
                         BatchNormalization, Flatten, Reshape
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras import regularizers

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dj = pd.read_csv("/content/gdrive/My Drive/SOP/BSE-Sensex-data.csv")
news = pd.read_csv("/content/gdrive/My Drive/SOP/done.csv", na_values = '?' )

## Inspect the data

In [0]:
dj.head()

,Unnamed: 0,Date,Open,High,Low,Close,Movement
0,0,2019-04-22,39158.22,39158.22,38585.65,38645.18,1
1,1,2019-04-18,39420.04,39487.45,39083.16,39140.28,1
2,2,2019-04-16,39040.30,39364.34,39038.81,39275.64,1
3,3,2019-04-15,38805.54,38976.58,38780.08,38905.84,0
4,4,2019-04-12,38692.06,38818.87,38554.79,38767.11,0


In [0]:
dj.drop(columns=['Unnamed: 0'], inplace = True)

In [0]:
dj.isnull().sum()

Date           0
Open        2380
High        2382
Low         2382
Close          0
Movement       0
dtype: int64

In [0]:
news.isnull().sum()

URL           0
Date        521
Title       521
Keywords    521
dtype: int64

In [0]:
news.dropna(inplace=True)
dj.dropna(inplace = True)
news.drop(columns=['URL', 'Keywords'], inplace = True)
news.head()

,Date,Title
0,2019-02-20,"GST Council meet: Relief for exporters, tax ex..."
1,2018-05-28,Índia Inc's days of cheap dollar loans may be ...
2,2017-09-25,"1,000 bank branches open Aadhaar updation cent..."
3,2019-02-08,"1.2 bn Indians, but just 150,000 declared inco..."
4,2018-06-18,"1.21 bn Aadhaar generated till June 16, UMANG ..."


In [0]:
print(dj.shape)
print(news.shape)

(6833, 6)
(18899, 2)


In [0]:
# Compare the number of unique dates. We want matching values.
print(len(set(dj.Date)))
print(len(set(news.Date)))

6833
1653


In [0]:
df = dj.set_index('Date')

In [0]:
def nearest(items, pivot):
    return min(items, key=lambda x: (x - pivot).days >0)

In [0]:
import dateutil

In [0]:
j = list(dj.Date)

In [0]:
k=[]
for d in j:
    k.append(dateutil.parser.parse(d).date())

In [0]:
(k[1] - k[0]).days

-4

In [0]:
nearest(k,dateutil.parser.parse(news['Date'][0]).date())

datetime.date(2019, 2, 20)

In [0]:
for n in range(len(news)):
    news.iloc[n,0] = str(nearest(k,dateutil.parser.parse(news.iloc[n,0]).date()))        

In [0]:
news.head()

,Date,Title
0,2019-02-20,"GST Council meet: Relief for exporters, tax ex..."
1,2018-05-28,Índia Inc's days of cheap dollar loans may be ...
2,2017-09-25,"1,000 bank branches open Aadhaar updation cent..."
3,2019-02-08,"1.2 bn Indians, but just 150,000 declared inco..."
4,2018-06-18,"1.21 bn Aadhaar generated till June 16, UMANG ..."


In [0]:
# Remove the extra dates that are in news
news = news[news.Date.isin(dj.Date)]

In [0]:
print(len(set(dj.Date)))
print(len(set(news.Date)))

6833
1190


In [0]:
news.shape

(18899, 2)

In [0]:
news.to_csv('done1.csv')

In [0]:
dj.head()

,Date,Open,High,Low,Close,Movement
0,2019-04-22,39158.22,39158.22,38585.65,38645.18,1
1,2019-04-18,39420.04,39487.45,39083.16,39140.28,1
2,2019-04-16,39040.30,39364.34,39038.81,39275.64,1
3,2019-04-15,38805.54,38976.58,38780.08,38905.84,0
4,2019-04-12,38692.06,38818.87,38554.79,38767.11,0


In [0]:
# Calculate the difference in opening prices between the following and current day.
# The model will try to predict how much the Open value will change beased on the news.
#dj = dj.set_index('Date').diff(periods=1)
#dj['Date'] = dj.index
#dj = dj1.reset_index(drop=True)
# Remove unneeded features
dj = dj.drop(['High','Low','Open','Close'], 1)

In [0]:
dj.head()

,Date,Movement
0,2019-04-22,1
1,2019-04-18,1
2,2019-04-16,1
3,2019-04-15,0
4,2019-04-12,0


In [0]:
# Remove top row since it has a null value.
#dj = dj[dj.Open.notnull()]

In [0]:
# Check if there are any more null values.
dj.isnull().sum()

Date        0
Movement    0
dtype: int64

In [0]:
dj.rename({'Movement':'Open'}, axis = 1,inplace = True)

In [0]:
dj.head()

,Date,Open
0,2019-04-22,1
1,2019-04-18,1
2,2019-04-16,1
3,2019-04-15,0
4,2019-04-12,0


In [0]:
# Create a list of the opening prices and their corresponding daily headlines from the news
price = []
headlines = []

for row in dj.iterrows():
    daily_headlines = []
    date = row[1]['Date']
    
    for row_ in news[news.Date==date].iterrows():
        daily_headlines.append(row_[1]['Title'])
    
    # Track progress
    if len(daily_headlines)>0 :
        headlines.append(daily_headlines)
        price.append(row[1]['Open'])
    if len(price) % 500 == 0:
        print(len(price))

0
0
0
0
0
0
0
500
1000


In [0]:
# Compare lengths to ensure they are the same
print(len(price))
print(len(headlines))

1190
1190


In [0]:
headlines

[['Sports Authority of India to be renamed: Rajyavardhan Singh Rathore',
  'Take over the defaulting sugar mills: Balyan to UP govt'],
 ['Corporate Affairs and Government Policy News',
  'Corporate Affairs and Government Policy News'],
 ['Karnataka urges duty hike on imported iron ore, pellets to aid local miners',
  'Corporate Affairs and Government Policy News'],
 ["India's 'power' conundrum: Deeply connected but not yet electrified"],
 ['Adityanath govt banking on MSME scheme in UP to counter unemployment',
  'Centre issue tenders for developing 30,000 MW of renewable projects',
  'Collapse in farm jobs reason behind rising unemployment, shows NSSO data',
  'Masala bond market revives as rupee gains and tax cut lures buyers',
  "Rahul's income plan for poor may cost more than Budget 2019's subsidy bill",
  "Mixed scorecard: After good start, Modi govt's reform measures losing steam",
  'Regulatory board for gold gets Finance Ministry nod, ball in PMO court',
  "Mission Shakti: India

In [0]:
# Compare the number of headlines for each day
print(max(len(i) for i in headlines))
print(min(len(i) for i in headlines))
#print(np.mean(len(i) for i in headlines))

136
1


In [0]:
sum(len(i) for i in headlines)/len(headlines)

15.881512605042017

In [0]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [0]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Clean the headlines
clean_headlines = []

for daily_headlines in headlines:
    clean_daily_headlines = []
    for headline in daily_headlines:
        clean_daily_headlines.append(clean_text(headline))
    clean_headlines.append(clean_daily_headlines)

In [0]:
# Take a look at some headlines to ensure everything was cleaned well
clean_headlines[789]

['coal linkage policy government depts power sector tied knots',
 'global economy future tense',
 'new crop insurance scheme cover 50 farmers']

In [0]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

for date in clean_headlines:
    for headline in date:
        for word in headline.split():
            if word not in word_counts:
                word_counts[word] = 1
            else:
                word_counts[word] += 1
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 13688


In [0]:
# Load GloVe's embeddings
embeddings_index = {}
with open('/Users/parththacker/Downloads/glove.840B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

FileNotFoundError: ignored

In [0]:
# Find the number of words that are missing from GloVe, and are used more than our threshold.
missing_words = 0
threshold = 10

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from GloVe:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from GloVe: 2473
Percent of words that are missing from vocabulary: 18.07%


In [0]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total Number of Unique Words:", len(word_counts))
print("Number of Words we will use:", len(vocab_to_int))
print("Percent of Words we will use: {}%".format(usage_ratio))

Total Number of Unique Words: 13688
Number of Words we will use: 2653
Percent of Words we will use: 19.38%


In [0]:
# Need to use 300 for embedding dimensions to match GloVe's vectors.
embedding_dim = 300

nb_words = len(vocab_to_int)
# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim))
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in GloVe, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

2653


Note: The embeddings will be updated as the model trains, so our new 'random' embeddings will be more accurate by the end of training. This is also why we want to only use words that appear at least 10 times. By having the model see the word numerous timesm it will be better able to understand what it means. 

In [0]:
# Change the text from words to integers
# If word is not in vocab, replace it with <UNK> (unknown)
word_count = 0
unk_count = 0

int_headlines = []

for date in clean_headlines:
    int_daily_headlines = []
    for headline in date:
        int_headline = []
        for word in headline.split():
            word_count += 1
            if word in vocab_to_int:
                int_headline.append(vocab_to_int[word])
            else:
                int_headline.append(vocab_to_int["<UNK>"])
                unk_count += 1
        int_daily_headlines.append(int_headline)
    int_headlines.append(int_daily_headlines)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 162689
Total number of UNKs in headlines: 26485
Percent of words that are UNK: 16.28%


In [0]:
# Find the length of headlines
lengths = []
for date in int_headlines:
    for headline in date:
        lengths.append(len(headline))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [0]:
lengths.describe()

,counts
count,18899.000000
mean,8.608339
std,1.956195
min,1.000000
25%,7.000000
50%,9.000000
75%,10.000000
max,17.000000


In [0]:
# Limit the length of a day's news to 200 words, and the length of any headline to 16 words.
# These values are chosen to not have an excessively long training time and 
# balance the number of headlines used and the number of words from each headline.
max_headline_length = 16
max_daily_length = 2000
pad_headlines = []

for date in int_headlines:
    pad_daily_headlines = []
    for headline in date:
        # Add headline if it is less than max length
        if len(headline) <= max_headline_length:
            for word in headline:
                pad_daily_headlines.append(word)
        # Limit headline if it is more than max length  
        else:
            headline = headline[:max_headline_length]
            for word in headline:
                pad_daily_headlines.append(word)
    
    # Pad daily_headlines if they are less than max length
    if len(pad_daily_headlines) < max_daily_length:
        for i in range(max_daily_length-len(pad_daily_headlines)):
            pad = vocab_to_int["<PAD>"]
            pad_daily_headlines.append(pad)
    # Limit daily_headlines if they are more than max length
    else:
        pad_daily_headlines = pad_daily_headlines[:max_daily_length]
    pad_headlines.append(pad_daily_headlines)

In [0]:
# Normalize opening prices (target values)
#max_price = max(price)
#min_price = min(price)
#mean_price = np.mean(price)
def normalize(price):
    return ((price-min_price)/(max_price-min_price))

In [0]:
norm_price = price
#for p in price:
#    norm_price.append(normalize(p))

In [0]:
# Check that normalization worked well
print(min(norm_price))
print(max(norm_price))
print(np.mean(norm_price))

0
1
0.46638655462184875


In [0]:
# Split data into training and testing sets.
# Validating data will be created during training.
x_train, x_test, y_train, y_test = train_test_split(pad_headlines, norm_price, test_size = 0.15, random_state = 2)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
# Check the lengths
print(len(x_train))
print(len(x_test))

1011
179


In [0]:
np.save('x_train', x_train)
np.save('x_test', x_test)
np.save('y_train', y_train)
np.save('y_test', y_test)



NameError: ignored

In [0]:
x_train = np.load('/content/gdrive/My Drive/SOP/x_train.npy')
x_test = np.load('/content/gdrive/My Drive/SOP/x_test.npy')
y_train = np.load('/content/gdrive/My Drive/SOP/y_train.npy')
y_test = np.load('/content/gdrive/My Drive/SOP/y_test.npy')

In [0]:
x_train.shape

(1011, 2000)

In [0]:
filter_length1 = 3
filter_length2 = 5
dropout = 0.5
learning_rate = 0.001
weights = TruncatedNormal(mean=0.0, stddev=0.1, seed=2)
nb_filter = 16
rnn_output_size = 128
hidden_dims = 128
wider = True
deeper = True

if wider == True:
    nb_filter *= 2
    rnn_output_size *= 2
    hidden_dims *= 2


def build_model():
    
    model1 = Sequential()
    
    model1.add(Embedding(nb_words, 
                         embedding_dim,
                         weights=[word_embedding_matrix], 
                         input_length=max_daily_length))
    model1.add(Dropout(dropout))
    
    model1.add(Convolution1D(filters = nb_filter, 
                             kernel_size = filter_length1, 
                             padding = 'same',
                            activation = 'relu'))
    model1.add(Dropout(dropout))
    
    if deeper == True:
        model1.add(Convolution1D(filters = nb_filter, 
                                 kernel_size = filter_length1, 
                                 padding = 'same',
                                activation = 'relu'))
        model1.add(Dropout(dropout))
    
    model1.add(LSTM(rnn_output_size, 
                   activation=None,
                   kernel_initializer=weights,
                   dropout = dropout))
    
    ####

    model2 = Sequential()
    
    model2.add(Embedding(nb_words, 
                         embedding_dim,
                         weights=[word_embedding_matrix], 
                         input_length=max_daily_length))
    model2.add(Dropout(dropout))
    
    
    model2.add(Convolution1D(filters = nb_filter, 
                             kernel_size = filter_length2, 
                             padding = 'same',
                             activation = 'relu'))
    model2.add(Dropout(dropout))
    
    if deeper == True:
        model2.add(Convolution1D(filters = nb_filter, 
                                 kernel_size = filter_length2, 
                                 padding = 'same',
                                 activation = 'relu'))
        model2.add(Dropout(dropout))
    
    model2.add(LSTM(rnn_output_size, 
                    activation=None,
                    kernel_initializer=weights,
                    dropout = dropout))
    
    ####

    model = Sequential()

    model.add(Merge([model1, model2], mode='concat'))
    
    model.add(Dense(hidden_dims, kernel_initializer=weights))
    model.add(Dropout(dropout))
    
    if deeper == True:
        model.add(Dense(hidden_dims//2, kernel_initializer=weights))
        model.add(Dropout(dropout))

    model.add(Dense(1, activation = 'softmax',
                    kernel_initializer = weights,
                    name='output'))

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=learning_rate,clipvalue=1.0))
    return model

In [0]:
# Use grid search to help find a better model
for deeper in [True]:
    for wider in [True]:
        for learning_rate in [0.001]:
            for dropout in [0.3, 0.5]:
                model = build_model()
                print()
                print("Current model: Deeper={}, Wider={}, LR={}, Dropout={}".format(
                    deeper,wider,learning_rate,dropout))
                print()
                save_best_weights = 'question_pairs_weights_deeper={}_wider={}_lr={}_dropout={}.h5'.format(
                    deeper,wider,learning_rate,dropout)

                callbacks = [ModelCheckpoint(save_best_weights, monitor='val_loss', save_best_only=True),
                             EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto'),
                             ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)]

                history = model.fit([x_train,x_train],
                                    y_train,
                                    batch_size=100,
                                    epochs=10,
                                    validation_split=0.15,
                                    verbose=True,
                                    shuffle=True,
                                    callbacks = callbacks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



Current model: Deeper=True, Wider=True, LR=0.001, Dropout=0.3

Train on 859 samples, validate on 152 samples
Epoch 1/10
859/859 [==============================] - 63s - loss: 8.5372 - val_loss: 8.4956
Epoch 2/10
859/859 [==============================] - 51s - loss: 8.5372 - val_loss: 8.4956
Epoch 3/10
200/859 [=====>........................] - ETA: 36s - loss: 7.9712

KeyboardInterrupt: ignored

In [0]:
# Make predictions with the best weights
deeper=False
wider=True
dropout=0.5
learning_Rate = 0.001
# Need to rebuild model in case it is different from the model that was trained most recently.
model = build_model()

model.load_weights('./question_pairs_weights_deeper={}_wider={}_lr={}_dropout={}.h5'.format(
                    deeper,wider,learning_rate,dropout))
predictions = model.predict([x_test,x_test], verbose = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


179/179 [==============================] - 9s     


In [0]:
# Compare testing loss to training and validating los
acc(y_test, predictions)

0.4748603351955307